## Imports

In [ ]:
import os
import sys

import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, classification_report, make_scorer
from sklearn.model_selection import GridSearchCV, StratifiedKFold, KFold

dir_parts = os.getcwd().split(os.path.sep)
root_index = dir_parts.index('MyML-B')
root_path = os.path.sep.join(dir_parts[:root_index + 1])
sys.path.append(root_path + '/code/')
from data.data_config import Dataset
from data.data_utils import load_monk, load_cup, store_monk_result, store_cup_result
from hyperparameter_tuning import grid_search_top_configs
from training.metrics import mean_euclidean_error

%load_ext autoreload
%autoreload 2

# Random Forest
In this notebook we test a **RandomForestClassifier** and a **RandomForestRegressor** w.r.t. the tasks at hand, i.e. the three MONK's problems and the CUP dataset respectively.

In [ ]:
MODEL_NAME = 'RandomForest'

## Path

In [ ]:
# Directories
results_dir = root_path + '/results/' + MODEL_NAME

# Filepaths (MONK)
m1_dev_path, m1_test_path = Dataset.MONK_1.dev_path, Dataset.MONK_1.test_path # MONK 1
m2_dev_path, m2_test_path = Dataset.MONK_2.dev_path, Dataset.MONK_2.test_path # MONK 2
m3_dev_path, m3_test_path = Dataset.MONK_3.dev_path, Dataset.MONK_3.test_path # MONK 3

# Filepaths (CUP)
cup_dev_path, cup_test_path = Dataset.CUP.dev_path, Dataset.CUP.test_path

# MONK-1

In [ ]:
# Load MONK-1
x_dev_m1, y_dev_m1, x_test_m1, y_test_m1 = load_monk(m1_dev_path, m1_test_path)

First of all, we define the grid search spaces for the RandomForestClassifier that we're going to use for all three MONK's problems.

In [ ]:
# hparams grid for grid search
rfc_hparams_spaces = {
    'n_estimators': [25, 50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'max_features': ['auto', 'sqrt'],
    'criterion': ['gini', 'entropy'],
    'bootstrap': [True, False],
}

## Grid search
Grid search is a simple hyper-parameter tuning techniques useful for finding the best configuration for a specific ML model. It involves:
- defining a grid of hparams values
- systematically test all possible combinations

Other approaches include Random search or Bayesian optimization.

In [ ]:
# perform grid search with KFold
grid_search_m1 = GridSearchCV(
    RandomForestClassifier(random_state=69),
    param_grid=rfc_hparams_spaces,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=69),
    scoring='accuracy',
    verbose=1
)

grid_search_m1.fit(x_dev_m1, y_dev_m1)

In [ ]:
best_configs_m1 = grid_search_top_configs(grid_search_m1.cv_results_) # top k config

In [ ]:
# Create a RandomForest with the best hparams
rfc = RandomForestClassifier(**grid_search_m1.best_params_)

# Train the model
rfc.fit(x_dev_m1, y_dev_m1)

In [ ]:
print('-- DEVELOPMENT --')
print(classification_report(y_dev_m1, rfc.predict(x_dev_m1)))

In [ ]:
print('-- TEST --')
report_test_m1 = classification_report(y_test_m1, rfc.predict(x_test_m1))
print(report_test_m1)

## Store results

In [ ]:
store_monk_result(results_dir + '/MONK1/', best_configs_m1, report_test_m1)

# MONK-2

# MONK-3

# CUP

In [ ]:
# Load CUP
x_dev_cup, y_dev_cup, x_test_cup = load_cup(cup_dev_path, cup_test_path)

Here, we define the grid search spaces for the RandomForestRegression for the CUP dataset.

In [ ]:
rfr_hparams_spaces = {
    'n_estimators': [25, 50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'max_features': ['auto', 'sqrt'],
    'criterion': ['squared_error'],
    'bootstrap': [True, False],
}

## Grid search

In [ ]:
# perform grid search with KFold
grid_search_cup = GridSearchCV(
    RandomForestRegressor(random_state=69),
    param_grid=rfr_hparams_spaces,
    cv=KFold(n_splits=5, shuffle=True, random_state=69),
    scoring=make_scorer(mean_euclidean_error, greater_is_better=False),
    verbose=1
)

grid_search_cup.fit(x_dev_cup, y_dev_cup)

In [ ]:
best_configs_cup = grid_search_top_configs(grid_search_cup.cv_results_) # top k config

In [ ]:
# Create a RandomForest with the best hparams
rfr = RandomForestRegressor(**grid_search_cup.best_params_)

# Train the model
rfr.fit(x_dev_cup, y_dev_cup)

In [ ]:
print('-- DEVELOPMENT --')
dev_mee_cup = mean_euclidean_error(y_dev_cup, rfr.predict(x_dev_cup))
print(dev_mee_cup)

In [ ]:
# Blind test set predictions
test_preds_cup = rfr.predict(x_test_cup)

## Store Result

In [ ]:
store_cup_result(results_dir + '/CUP/', best_configs_cup, dev_mee_cup, test_preds_cup)